In [1]:
using Revise
using PauliPropagation

Precompiling PauliPropagation
  ✓ PauliPropagation
  1 dependency successfully precompiled in 1 seconds. 10 already precompiled.


In [2]:
nq = 8

8

In [3]:
symbs = [:I for _ in 1:nq]
symbs[round(Int, nq/2)] = :Z   # as symbol. Also works but is slower.

obsint = symboltoint(symbs)  # for performance we work with bitoperations
show(obsint, nq)

IIIZIIII


In [4]:
nl = 4
topo = bricklayertopology(nq; periodic=false)
# topo = get2dtopology(4, 4)
circ = hardwareefficientcircuit(nq, nl; topology=topo)
fastcirc = tofastgates(circ)
m = length(fastcirc)

124

In [5]:
using Random
Random.seed!(42)
thetas = randn(m);

In [6]:
W = Inf;                   # maximal operator weight.
min_abs_coeff = 0;          # neglect small coefficients. Only for numerical and hybrid PP.

#### Numerical Pauli Propagation
Propagates numerical coefficients.

In [7]:
@time dnum = mergingbfs(fastcirc, obsint, thetas; max_weight=W, min_abs_coeff=min_abs_coeff);
@show length(dnum)   # number of unique Pauli ops
evalagainstzero(dnum) # expectation

  0.423178 seconds (686.31 k allocations: 50.730 MiB, 5.14% gc time, 73.99% compilation time)
length(dnum) = 64452


0.03270457825165109

In [8]:
show(dnum)

Dict{UInt16, Float64} with 64452 entries:
  XZIIXXZX => 0.0006550291547293156
  IIIXIIZY => -4.859870209231609e-7
  YZZIYXZX => -0.00018262676667615688
  ZXYYYXII => 0.005961755782095122
  IZYZIIXY => 0.0009020582404861559
  XXIIYZXI => 0.004495755588131953
  IZZXIZYZ => -0.004345900860351217
  YYXIYYYZ => 0.0018501555717484876
  YZIXXZII => -0.0035136671430621674
  IXIYIIZX => 0.003479054477339053
  YIYZYIZZ => -1.4667805954951649e-5
  IYZZZYYX => 0.0017093807533506405
  XZZXZZYZ => -0.0023265852263534155
  XIZZXXZZ => 0.0004591710404569854
  ZZXZZXII => -0.004155086459294072
  ZZYXYYYY => -0.0007206204203938528
  XXXYIYXZ => -0.0012399669774994
  XZYXIYYZ => -0.010345152142423328
  IIIXZZXZ => -3.842916397610179e-5
  XXXXZZXY => -0.0009745454859420705
  ⋮


#### Hybrid Pauli Propagation
Propagates numerical coefficients, but can but truncated like the surrogate.

In [9]:
max_freq = Inf   # max frequency, i.e., max number of sines and cosines per path

Inf

In [10]:
@time dhyb = mergingbfs(fastcirc, obsint, NumericPathProperties(1.0), thetas; max_weight=W, max_freq=Inf, min_abs_coeff=min_abs_coeff);
@show length(dhyb)
evalagainstzero(dhyb)

  0.266300 seconds (1.25 M allocations: 68.696 MiB, 2.46% gc time, 54.41% compilation time)
length(dhyb) = 64452


0.03270457825165109

In [11]:
show(dhyb)

Dict{UInt16, NumericPathProperties} with 64452 entries:
  XZIIXXZX => NumericPathProperties(0.0006550291547293156, nsins=8, ncos=5, freq=26)
  IIIXIIZY => NumericPathProperties(-4.859870209231609e-7, nsins=7, ncos=7, freq=28)
  YZZIYXZX => NumericPathProperties(-0.00018262676667615688, nsins=8, ncos=4, freq=26)
  ZXYYYXII => NumericPathProperties(0.005961755782095122, nsins=5, ncos=5, freq=23)
  IZYZIIXY => NumericPathProperties(0.0009020582404861559, nsins=6, ncos=8, freq=29)
  XXIIYZXI => NumericPathProperties(0.004495755588131953, nsins=9, ncos=8, freq=23)
  IZZXIZYZ => NumericPathProperties(-0.004345900860351217, nsins=7, ncos=6, freq=28)
  YYXIYYYZ => NumericPathProperties(0.0018501555717484876, nsins=7, ncos=6, freq=27)
  YZIXXZII => NumericPathProperties(-0.0035136671430621674, nsins=7, ncos=5, freq=21)
  IXIYIIZX => NumericPathProperties(0.003479054477339053, nsins=7, ncos=6, freq=27)
  YIYZYIZZ => NumericPathProperties(-1.4667805954951649e-5, nsins=7, ncos=4, freq=27)
  IYZZZY

#### Pauli Propagation Surrogate
Builds a graph that can later be evaluated.

In [12]:
@time dsym = mergingbfs(circ, operatortopathdict(obsint), zeros(m); max_weight=W, max_freq=max_freq);
@show length(dsym)

final_nodes = collect(pth.coeff for (obs, pth) in zerofilter(dsym));
final_eval_node = PauliGateNode(parents=final_nodes, trig_inds=zeros(Int, length(final_nodes)), signs=ones(length(final_nodes)), param_idx=1, cummulative_value=0.0);
resetnodes(final_eval_node)
resetnodes(final_eval_node)
@time eval_list = gettraceevalorder(final_eval_node, zeros(m));
length(eval_list)  # The list of all nodes. The order is such that one can savely be evaluated after the other.

  1.417593 seconds (15.90 M allocations: 857.028 MiB, 33.99% gc time, 19.46% compilation time)
length(dsym) = 64452
  0.107263 seconds (32.10 k allocations: 7.431 MiB, 30.67% compilation time)


399608

In [13]:
@time expectation(eval_list, thetas)    # This is actually not always faster than numerical propagation, but in interesting cases it is by a lot.
                                        # making this always at least as fast is work in progress. Graph traversal is hard.

  0.080449 seconds (62.00 k allocations: 4.309 MiB, 42.61% compilation time)


0.032704578251651115

In [20]:
show(setelement!(op, 1, 0x02))

YXXXXIII


In [42]:
ZZpihalf_relations = Dict(  # with transpose || with permutedims
    (:I, :I) => (1, :I, :I),
    (:I, :X) => (1, :Z, :Y),
    (:I, :Y) => (-1, :Z, :X),
    (:I, :Z) => (1, :I, :Z),
    (:X, :I) => (1, :Y, :Z),
    (:X, :X) => (1, :X, :X),
    (:X, :Y) => (1, :X, :Y),
    (:X, :Z) => (1, :Y, :I),
    (:Y, :I) => (-1, :X, :Z),
    (:Y, :X) => (1, :Y, :X),
    (:Y, :Y) => (1, :Y, :Y),
    (:Y, :Z) => (-1, :X, :I),
    (:Z, :I) => (1, :Z, :I),
    (:Z, :X) => (1, :I, :Y),
    (:Z, :Y) => (-1, :I, :X),
    (:Z, :Z) => (1, :Z, :Z),
)

Dict{Tuple{Symbol, Symbol}, Tuple{Int64, Symbol, Symbol}} with 16 entries:
  (:X, :Z) => (1, :Y, :I)
  (:Y, :Z) => (-1, :X, :I)
  (:I, :X) => (1, :Z, :Y)
  (:X, :X) => (1, :X, :X)
  (:Y, :Y) => (1, :Y, :Y)
  (:Y, :I) => (-1, :X, :Z)
  (:I, :I) => (1, :I, :I)
  (:Z, :Z) => (1, :Z, :Z)
  (:X, :Y) => (1, :X, :Y)
  (:I, :Y) => (-1, :Z, :X)
  (:X, :I) => (1, :Y, :Z)
  (:Z, :Y) => (-1, :I, :X)
  (:Z, :I) => (1, :Z, :I)
  (:Y, :X) => (1, :Y, :X)
  (:I, :Z) => (1, :I, :Z)
  (:Z, :X) => (1, :I, :Y)

In [21]:
CNOT_relations = Dict(
    (:I, :I) => (1, :I, :I),
    (:I, :X) => (1, :I, :X),
    (:I, :Y) => (1, :Z, :Y),
    (:I, :Z) => (1, :Z, :Z),
    (:X, :I) => (1, :X, :X),
    (:X, :X) => (1, :X, :I),
    (:X, :Y) => (1, :Y, :Z),
    (:X, :Z) => (-1, :Y, :Y),
    (:Y, :I) => (1, :Y, :X),
    (:Y, :X) => (1, :Y, :I),
    (:Y, :Y) => (-1, :X, :Z),
    (:Y, :Z) => (1, :X, :Y),
    (:Z, :I) => (1, :Z, :I),
    (:Z, :X) => (1, :Z, :X),
    (:Z, :Y) => (1, :I, :Y),
    (:Z, :Z) => (1, :I, :Z),
)

Dict{Tuple{Symbol, Symbol}, Tuple{Int64, Symbol, Symbol}} with 16 entries:
  (:X, :Z) => (-1, :Y, :Y)
  (:Y, :Z) => (1, :X, :Y)
  (:I, :X) => (1, :I, :X)
  (:X, :X) => (1, :X, :I)
  (:Y, :Y) => (-1, :X, :Z)
  (:Y, :I) => (1, :Y, :X)
  (:I, :I) => (1, :I, :I)
  (:Z, :Z) => (1, :I, :Z)
  (:X, :Y) => (1, :Y, :Z)
  (:I, :Y) => (1, :Z, :Y)
  (:X, :I) => (1, :X, :X)
  (:Z, :Y) => (1, :I, :Y)
  (:Z, :I) => (1, :Z, :I)
  (:Y, :X) => (1, :Y, :I)
  (:I, :Z) => (1, :Z, :Z)
  (:Z, :X) => (1, :Z, :X)

In [37]:
new_rels = Dict()
for (k, v) in CNOT_relations
    new_rels[symboltoint(collect(k))] = (v[1], symboltoint([v[2], v[3]]))
end

In [33]:
new_rels

Dict{Any, Any} with 16 entries:
  0x05 => (1, 0x01)
  0x0c => (1, 0x0f)
  0x08 => (1, 0x0b)
  0x01 => (1, 0x05)
  0x00 => (1, 0x00)
  0x06 => (1, 0x02)
  0x0b => (1, 0x08)
  0x09 => (1, 0x0e)
  0x0e => (1, 0x09)
  0x03 => (1, 0x03)
  0x07 => (1, 0x07)
  0x0d => (-1, 0x0a)
  0x04 => (1, 0x04)
  0x0f => (1, 0x0c)
  0x02 => (1, 0x06)
  0x0a => (-1, 0x0d)

In [38]:
default_clifford_map[:CNOT][0x03]

(1, 0x06)

In [39]:
print([new_rels[ii] for ii in 0:15])

Tuple{Int64, UInt8}[(1, 0x00), (1, 0x05), (1, 0x06), (1, 0x03), (1, 0x04), (1, 0x01), (1, 0x02), (1, 0x07), (1, 0x0b), (1, 0x0e), (-1, 0x0d), (1, 0x08), (1, 0x0f), (-1, 0x0a), (1, 0x09), (1, 0x0c)]

In [4]:
show(obsint)

IIIZIIII


In [65]:
g = CliffordGate(:CNOT, [1, 3])

CliffordGate(:CNOT, [1, 3])

In [97]:
op = symboltoint([:Y, :I, :Y, :I])

0x22

In [98]:
o, _ = apply(g, op)

(0x31, -1.0)

In [99]:
show(o)

XIZI


In [100]:
lookup_op = PauliPropagation._extractlookupop(op, g.qinds)
show(lookup_op)

YYII


In [101]:
show(default_clifford_map[:CNOT][lookup_op+1][2])

XZII


In [102]:
CNOT_relations = Dict(
    (:I, :I) => (1, :I, :I),
    (:I, :X) => (1, :I, :X),
    (:I, :Y) => (1, :Z, :Y),
    (:I, :Z) => (1, :Z, :Z),
    (:X, :I) => (1, :X, :X),
    (:X, :X) => (1, :X, :I),
    (:X, :Y) => (1, :Y, :Z),
    (:X, :Z) => (-1, :Y, :Y),
    (:Y, :I) => (1, :Y, :X),
    (:Y, :X) => (1, :Y, :I),
    (:Y, :Y) => (-1, :X, :Z),
    (:Y, :Z) => (1, :X, :Y),
    (:Z, :I) => (1, :Z, :I),
    (:Z, :X) => (1, :Z, :X),
    (:Z, :Y) => (1, :I, :Y),
    (:Z, :Z) => (1, :I, :Z),
)

Dict{Tuple{Symbol, Symbol}, Tuple{Int64, Symbol, Symbol}} with 16 entries:
  (:X, :Z) => (-1, :Y, :Y)
  (:Y, :Z) => (1, :X, :Y)
  (:I, :X) => (1, :I, :X)
  (:X, :X) => (1, :X, :I)
  (:Y, :Y) => (-1, :X, :Z)
  (:Y, :I) => (1, :Y, :X)
  (:I, :I) => (1, :I, :I)
  (:Z, :Z) => (1, :I, :Z)
  (:X, :Y) => (1, :Y, :Z)
  (:I, :Y) => (1, :Z, :Y)
  (:X, :I) => (1, :X, :X)
  (:Z, :Y) => (1, :I, :Y)
  (:Z, :I) => (1, :Z, :I)
  (:Y, :X) => (1, :Y, :I)
  (:I, :Z) => (1, :Z, :Z)
  (:Z, :X) => (1, :Z, :X)